# Networkx

* Пакет для Python для манипулирования графиками и их анализа
* Содержит множество стандатных алгоритмов для графов

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

## Создание графов

Networkx поддерживает неориентированные/ориентированные графы/мультиграфы (мультиграфы позволяют одной паре узлов иметь несколько ребер):
*  Неориентированный граф: `nx.Graph`
*  Ориентированный граф: `nx.DiGraph`
*  Неориентированный мультиграф: `nx.MultiGraph`
*  Ориентированный мультиграф: `nx.MultiDiGraph`

Networkx имеет отличный набор методов для отображения графов. Напишем функцию, которую будем использовать на вывода графов на экран

Классы графов имеют интерфейс для явного добавления ребер и узлов. Следующие команды, например, строят граф ниже

![graph 1](graph_1.png)

Направленный граф создается с помощью класса `nx.DiGraph`:

Мы также можем создать граф напрямую из списка ребер:

Опционально мы всегда можем сделать граф взвешенным. Для этого в метод `add_edge()` передается ключевое слово `weight`. Тоже можно сделать и используя метод `add_weighted_edges_from()`:

Названия узлов могут быть произвольными hashable. Мы также может добавлять произвольные аттрибуты в узлам и ребрам:

## Доступ к узлам и ребрам

Networkx предоставляет удобный интерфейс для доступа к узлам/ребрам и их аттрибутам, а также позволяет легко итерироваться по ним. Рассмотрим несколько популярных операций

Количество узлов в графе:

Количество ребер в графе:

Проверка, присутствует ли узел в графе:

Проверка, присутствует ли ребро в графе:

Итерация по узлам:

Итерация по ребрам:

Итерацией по ребрам вместе с аттрибутами:

## Доступ к соседям

Для начала рассмотрим случай ненаправленного графа.

Множество соседей данного узла можно получить, используя `G.neighbors(n)` или `G.adj[n]`. Например, итерация по соседям узла может выглядеть так:

Или так:

В направленных графах при рассмотрении соседей данного узла, то есть смежных узлов, нам важно разделять in-edges и out-edges. Для получения доступа к in-edges используется метод `G.predecessors()`, а для out-edges метод `G.successors()`.

Для нахождения степени вершины используется метод `G.degree(n)`, который реализован и для ненаправленных и для направленных графов. Для направленных графов существуют также отдельные методы для полустепеней захода и исхода (indegree и outdegree), `G.in_degree(n)` и `G.out_degree(n)` соответственно.

### Упражнение 1

Напишите функцию, вычисляющую среднюю степень соседей для каждого из узлов, у которых в принципе есть соседи

## Загрузка и сохранение графов

Наконец, мы можем сохранять графы в файлы и вычитывать их из них. Для простых задач мы можем использовать `adjlist` и `edgelist` форматы:
* `adjlist` является компактным представлением матрицы смежности. Он не подходит для графов с аттрибутами
* `edgelist` является списком ребер с их аттрибутами
* Для обоих методов названия узлов не должны включать пробелов

Методы `nx.read_adjlist()` и `nx.read_edgelist()` используются для чтения графов из файлов соответствующих форматов:

### Упражнение 2

Для n = 10, 20 и 30 найдите соответствующие значения p, при которых почти наверняка пройзодет невзвешенная перколяция в графе Эрдеша-Реньи G(n, p).

# Базовые характеристики графов

## Распределение степеней вершин

Если распределение степеней вершин имеет вид степенного закона $f(k) \sim k^{-\gamma}$ (распределение Парето), то говорят, что сеть является scale-free. Это название идет от свойства масштабной инвариантности степенного закона (инвариантность формы закона относительно $k \to \alpha$) - ни один из масштабов (имеется в виду масштаб связности, в нашем случае степени вершин) не доминирует в сети, и чем больше масштаб, тем больше вероятность его обнаружить. Это приводит к характерной особенности scale-free сетей - наличию в них хабов.

В качестве примера scale-free сети рассмотрим сеть наиболее загруженных аэропорта США. Действительно, хорошо известно, что некоторые аэропорты выполняют функцию хабов. Следовательно, мы должны суметь обнаружить scale-free феномен в сети аэропортов. Для этого сначала визуализируем сеть с помощью `plotly`

In [ ]:
def plot_network_via_plotly(G, pos, name):
    """
    Borrowed from https://plotly.com/python/network-graphs/
    """
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color="#888"),
        hoverinfo="none",
        mode="lines")

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode="markers",
        hoverinfo="text",
        marker=dict(
            showscale=True,
            colorscale="YlGnBu",
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title=dict(
                  text="Degree",
                  side="right"
                ),
                xanchor="left",
            ),
            line_width=2
        )
    )

    node_adjacencies = []
    node_text = []
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append(f"Degree: {len(adjacencies[1])}")

    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text

    fig = go.Figure(data=[edge_trace, node_trace],
                 layout=go.Layout(
                    title=dict(
                        font=dict(
                            size=16
                        )
                    ),
                    showlegend=False,
                    hovermode="closest",
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ dict(
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002 ) ],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                 )
    )
    fig.write_html(f"{name}.html")

Теперь убедимся, что распределение степеней вершин действительно описывается степенным законом для данного случая.

Далеко не все сети являются scale-free сетями. На самом деле интерес к ним появился только к концу 90-х, когда стало очевидно, что WWW сеть содержит в себе много хабов и, следовательно, ее свойства принципиально отличаются от таких случайных сетей, как, например, граф Эрдеша-Реньи. Действительно, можно убедиться, что распределение степеней вершин в графе Эрдеша-Реньи имеет отличную форму.